In [1]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="backtest.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[2410:MainThread](2021-10-27 14:43:43,812) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[2410:MainThread](2021-10-27 14:43:43,817) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[2410:MainThread](2021-10-27 14:43:44,023) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[2410:MainThread](2021-10-27 14:43:44,024) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[2410:MainThread](2021-10-27 14:47:08,709) INFO - qlib.timer - [log.py:113] - Time cost: 202.509s | Loading data Done
[2410:MainThread](2021-10-27 14:47:11,320) INFO - qlib.timer - [log.py:113] - Time cost: 2.248s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[2410:MainThread](2021-10-27 14:47:28,598) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[2410:MainThread](2021-10-27 14:47:28,601) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/us_158_nassp_epoch100 is not empty
[2410:MainThread](2021-10-27 14:47:28,603) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[2410:MainThread](2021-10-27 14:47:28,611) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[2410:MainThread](2021-10-27 14:47:28,613) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [126]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/us_158_nassp_epoch100/model.bin", map_location="cpu")["model"]

In [127]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [128]:
model.fitted = True

In [129]:
pred = model.predict(latest_dataset)

100%|████████████████████████████████████████████████████| 1405/1405 [03:15<00:00,  7.18it/s]
[2410:MainThread](2021-10-28 17:50:01,378) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.16716807435078926, 'MAE': 0.33334564110890963, 'IC': -0.0026503881868328895, 'ICIR': -0.023196184028815865}


In [130]:
import qlib
from qlib.data import D
qlib.init(provider_uri="/home/us_data", region="us")

df = D.features(D.instruments("nassp"), ["$open"], freq="day")

past_data = df.reset_index('instrument')

[2410:MainThread](2021-10-28 17:50:01,406) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[2410:MainThread](2021-10-28 17:50:01,411) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[2410:MainThread](2021-10-28 17:50:01,417) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[2410:MainThread](2021-10-28 17:50:01,419) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [131]:
reset_df = pred.reset_index('instrument')

In [132]:
from IPython.display import clear_output
import numpy as np
import os
import time

indexs = reset_df.index.drop_duplicates()
cnt = 0 

money = 100
current_buying = {}
top_k = 1
max_buying = 10
num_buy = 0
# fee = 0.07
# num_sell = 0
my_stack = None

profit = 0.0

data_dir = '/home/us_data_source/'
for idx in indexs:
    cnt+=1
    
    # if cnt < 800: continue
    # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
    # print(today_inform)
    if type(reset_df.loc[idx]['score']) is np.float32:
        stack = reset_df.loc[idx]
    else:
        stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)
        
    # st = time.time()
    current_buying_list = list(current_buying.keys())
    for cur_stock in current_buying_list:
        cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
        today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
        # if today_stock_price is np.nan:
        # print(np.isnan(today_stock_price))
        # else:
        current_buying[cur_stock][2] = today_stock_price
    
    # print("2", time.time() - st)
    
    my_stack = stack[stack['instrument'].isin(current_buying_list)]
    my_stack = my_stack.sort_values(by='score' , ascending=True)
    my_stack_list = list(my_stack['instrument'])
    # print(my_stack_list)
    # print(my_stack)
    for x in stack['instrument'][:top_k]:
        # print(x)
        if num_buy >= max_buying:
            last_stock = my_stack_list[-1]
            num_last_stock = current_buying[last_stock][0]
            
            num_buy -= 1
            # print(current_buying[last_stock])
            start, end = current_buying[last_stock][1:]
            if not (np.isnan(start) or np.isnan(end)):
                earn = (end - start) / start * money / max_buying
                money += earn
            
            if num_last_stock == 1:
                del current_buying[last_stock]
                my_stack_list = my_stack_list[:-1]
            else:
                current_buying[last_stock][0] -= 1
            
            # num_sell += 1
        if x not in current_buying:
            cur_stock = x
            cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
            today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
            current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
            num_buy +=1
        else:
            current_buying[x][0] += 1
            num_buy +=1
        # print(num_buy, current_buying)
        # input()
    
    # today_profit = 1
    # for cur in current_buying.values():
    #     num, start, end = cur
    #     profit = (end/start)**num
    #     today_profit *= profit
    # today_profit = today_profit**(1/num_buy)
    
    # if not np.isnan(today_profit):
    #     total_profit *= today_profit
    # total_profit = total_profit**(1/cnt)
    print("---------------------------------")
    print("day ", cnt, idx)
    print(num_buy, current_buying)
    # print("today_profit: ", today_profit)
    # print("total_profit: ", total_profit)
    print("current money: ", money)
    # print("------------------")
    # print(current_buying)
    # input()
    # clear_output()
    if cnt == 800:
        break

---------------------------------
day  1 2015-01-02 00:00:00
1 {'WELL': [1, 14.584031, 14.584031]}
current money:  100
---------------------------------
day  2 2015-01-05 00:00:00
2 {'WELL': [1, 14.584031, 14.710599], 'EA': [1, 2.23619, 2.23619]}
current money:  100
---------------------------------
day  3 2015-01-06 00:00:00
3 {'WELL': [1, 14.584031, 14.896613], 'EA': [2, 2.23619, 2.231905]}
current money:  100
---------------------------------
day  4 2015-01-07 00:00:00
4 {'WELL': [1, 14.584031, 15.151667], 'EA': [2, 2.23619, 2.203334], 'MYL': [1, 5.17332, 5.17332]}
current money:  100
---------------------------------
day  5 2015-01-08 00:00:00
5 {'WELL': [1, 14.584031, 15.358778], 'EA': [2, 2.23619, 2.238572], 'MYL': [1, 5.17332, 5.358621], 'APA': [1, 4.119941, 4.119941]}
current money:  100
---------------------------------
day  6 2015-01-09 00:00:00
6 {'WELL': [1, 14.584031, 15.203447], 'EA': [2, 2.23619, 2.288095], 'MYL': [1, 5.17332, 5.358621], 'APA': [1, 4.119941, 4.254522], '

KeyError: '2018-04-04'

In [133]:
from IPython.display import clear_output
import numpy as np
import os
import time

indexs = reset_df.index.drop_duplicates()
cnt = 0 

money = 100
current_buying = {}
top_k = 5
max_buying = 50
num_buy = 0
# fee = 0.07
# num_sell = 0
my_stack = None

profit = 0.0

data_dir = '/home/us_data_source/'
for idx in indexs:
    cnt+=1
    
    # if cnt < 800: continue
    # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
    # print(today_inform)
    if type(reset_df.loc[idx]['score']) is np.float32:
        stack = reset_df.loc[idx]
    else:
        stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)
        
    # st = time.time()
    current_buying_list = list(current_buying.keys())
    for cur_stock in current_buying_list:
        cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
        today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
        # if today_stock_price is np.nan:
        # print(np.isnan(today_stock_price))
        # else:
        current_buying[cur_stock][2] = today_stock_price
    
    # print("2", time.time() - st)
    
    my_stack = stack[stack['instrument'].isin(current_buying_list)]
    my_stack = my_stack.sort_values(by='score' , ascending=True)
    my_stack_list = list(my_stack['instrument'])
    # print(my_stack_list)
    # print(my_stack)
    for x in stack['instrument'][:top_k]:
        # print(x)
        if num_buy >= max_buying:
            last_stock = my_stack_list[-1]
            num_last_stock = current_buying[last_stock][0]
            
            num_buy -= 1
            # print(current_buying[last_stock])
            start, end = current_buying[last_stock][1:]
            if not (np.isnan(start) or np.isnan(end)):
                earn = (end - start) / start * money / max_buying
                money += earn
            
            if num_last_stock == 1:
                del current_buying[last_stock]
                my_stack_list = my_stack_list[:-1]
            else:
                current_buying[last_stock][0] -= 1
            
            # num_sell += 1
        if x not in current_buying:
            cur_stock = x
            cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
            today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
            current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
            num_buy +=1
        else:
            current_buying[x][0] += 1
            num_buy +=1
        # print(num_buy, current_buying)
        # input()
    
    # today_profit = 1
    # for cur in current_buying.values():
    #     num, start, end = cur
    #     profit = (end/start)**num
    #     today_profit *= profit
    # today_profit = today_profit**(1/num_buy)
    
    # if not np.isnan(today_profit):
    #     total_profit *= today_profit
    # total_profit = total_profit**(1/cnt)
    print("---------------------------------")
    print("day ", cnt, idx)
    print(num_buy, current_buying)
    # print("today_profit: ", today_profit)
    # print("total_profit: ", total_profit)
    print("current money: ", money)
    # print("------------------")
    # print(current_buying)
    # input()
    # clear_output()
    if cnt == 800:
        break

---------------------------------
day  1 2015-01-02 00:00:00
5 {'WELL': [1, 14.584031, 14.584031], 'PEAK': [1, 9.926856, 9.926856], 'BMY': [1, 1.78755, 1.78755], 'EW': [1, 15.538181, 15.538181], 'BDX': [1, 6.616302, 6.616302]}
current money:  100
---------------------------------
day  2 2015-01-05 00:00:00
10 {'WELL': [1, 14.584031, 14.710599], 'PEAK': [1, 9.926856, 9.994113], 'BMY': [2, 1.78755, 1.785444], 'EW': [1, 15.538181, 15.345455], 'BDX': [2, 6.616302, 6.603564], 'EA': [1, 2.23619, 2.23619], 'PDCO': [1, 4.908986, 4.908986], 'HOT': [1, 0.97561, 0.97561]}
current money:  100
---------------------------------
day  3 2015-01-06 00:00:00
15 {'WELL': [1, 14.584031, 14.896613], 'PEAK': [1, 9.926856, 10.207087], 'BMY': [3, 1.78755, 1.770706], 'EW': [1, 15.538181, 15.390304], 'BDX': [2, 6.616302, 6.674319], 'EA': [2, 2.23619, 2.231905], 'PDCO': [2, 4.908986, 4.920128], 'HOT': [1, 0.97561, 0.97561], 'ROST': [1, 24.003069, 24.003069], 'BBBY': [1, 4.405756, 4.405756]}
current money:  100
-

In [137]:
from IPython.display import clear_output
import numpy as np
import os
import time

indexs = reset_df.index.drop_duplicates()
cnt = 0 

money = 100
current_buying = {}
top_k = 5
max_buying = 30
num_buy = 0
loss_cut = 0.95
# profit_cut = 1.05
# num_sell = 0
my_stack = None

profit = 0.0

data_dir = '/home/us_data_source/'
for idx in indexs:
    cnt+=1
    try:
        # if cnt < 800: continue
        # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
        # print(today_inform)
        if type(reset_df.loc[idx]['score']) is np.float32:
            stack = reset_df.loc[idx]
        else:
            stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)

        # st = time.time()
        current_buying_list = list(current_buying.keys())
        for cur_stock in current_buying_list:
            cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
            today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
            # if today_stock_price is np.nan:
            # print(np.isnan(today_stock_price))
            # else:
            current_buying[cur_stock][2] = today_stock_price

        # print("2", time.time() - st)

        my_stack = stack[stack['instrument'].isin(current_buying_list)]
        my_stack = my_stack.sort_values(by='score' , ascending=True)
        my_stack_list = list(my_stack['instrument'])
        # print(my_stack_list)
        # print(my_stack)
        for x in stack['instrument'][:top_k]:
            # print(x)
            if num_buy >= max_buying:
                last_stock = my_stack_list[-1]
                num_last_stock = current_buying[last_stock][0]

                num_buy -= 1
                # print(current_buying[last_stock])
                start, end = current_buying[last_stock][1:]
                if not (np.isnan(start) or np.isnan(end)):
                    earn = (end - start) / start * money / max_buying
                    money += earn

                if num_last_stock == 1:
                    del current_buying[last_stock]
                    my_stack_list = my_stack_list[:-1]
                else:
                    current_buying[last_stock][0] -= 1

                # num_sell += 1
            if x not in current_buying:
                cur_stock = x
                cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
                today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
                current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
                num_buy +=1
            else:
                current_buying[x][0] += 1
                num_buy +=1
            # print(num_buy, current_buying)
            # input()

        # today_profit = 1
        # for cur in current_buying.values():
        #     num, start, end = cur
        #     profit = (end/start)**num
        #     today_profit *= profit
        # today_profit = today_profit**(1/num_buy)

        # if not np.isnan(today_profit):
        #     total_profit *= today_profit
        # total_profit = total_profit**(1/cnt)
        print("---------------------------------")
        print("day ", cnt, idx)
        print(num_buy, current_buying)
        # print("today_profit: ", today_profit)
        # print("total_profit: ", total_profit)
        print("current money: ", money)
        # print("------------------")
        # print(current_buying)
        # input()
        # clear_output()
    except:
        continue
    if cnt == 800:
        break

---------------------------------
day  1 2015-01-02 00:00:00
5 {'WELL': [1, 14.584031, 14.584031], 'PEAK': [1, 9.926856, 9.926856], 'BMY': [1, 1.78755, 1.78755], 'EW': [1, 15.538181, 15.538181], 'BDX': [1, 6.616302, 6.616302]}
current money:  100
---------------------------------
day  2 2015-01-05 00:00:00
10 {'WELL': [1, 14.584031, 14.710599], 'PEAK': [1, 9.926856, 9.994113], 'BMY': [2, 1.78755, 1.785444], 'EW': [1, 15.538181, 15.345455], 'BDX': [2, 6.616302, 6.603564], 'EA': [1, 2.23619, 2.23619], 'PDCO': [1, 4.908986, 4.908986], 'HOT': [1, 0.97561, 0.97561]}
current money:  100
---------------------------------
day  3 2015-01-06 00:00:00
15 {'WELL': [1, 14.584031, 14.896613], 'PEAK': [1, 9.926856, 10.207087], 'BMY': [3, 1.78755, 1.770706], 'EW': [1, 15.538181, 15.390304], 'BDX': [2, 6.616302, 6.674319], 'EA': [2, 2.23619, 2.231905], 'PDCO': [2, 4.908986, 4.920128], 'HOT': [1, 0.97561, 0.97561], 'ROST': [1, 24.003069, 24.003069], 'BBBY': [1, 4.405756, 4.405756]}
current money:  100
-

In [145]:
from IPython.display import clear_output
import numpy as np
import os
import time

indexs = reset_df.index.drop_duplicates()
cnt = 0 

money = 100
current_buying = {}
top_k = 5
max_buying = 20
num_buy = 0
loss_cut = 0.95
# profit_cut = 1.05
# num_sell = 0
my_stack = None

profit = 0.0

data_dir = '/home/us_data_source/'
for idx in indexs:
    cnt+=1
    try:
        # if cnt < 800: continue
        # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
        # print(today_inform)
        if type(reset_df.loc[idx]['score']) is np.float32:
            stack = reset_df.loc[idx]
        else:
            stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)

        # st = time.time()
        current_buying_list = list(current_buying.keys())
        for cur_stock in current_buying_list:
            cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
            today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
            # if today_stock_price is np.nan:
            # print(np.isnan(today_stock_price))
            # else:
            current_buying[cur_stock][2] = today_stock_price

        # print("2", time.time() - st)

        my_stack = stack[stack['instrument'].isin(current_buying_list)]
        my_stack = my_stack.sort_values(by='score' , ascending=True)
        my_stack_list = list(my_stack['instrument'])
        # print(my_stack_list)
        # print(my_stack)
        for x in stack['instrument'][:top_k]:
            # print(x)
            if num_buy >= max_buying:
                last_stock = my_stack_list[-1]
                num_last_stock = current_buying[last_stock][0]

                num_buy -= 1
                # print(current_buying[last_stock])
                start, end = current_buying[last_stock][1:]
                if not (np.isnan(start) or np.isnan(end)):
                    earn = (end - start) / start * money / max_buying
                    money += earn

                if num_last_stock == 1:
                    del current_buying[last_stock]
                    my_stack_list = my_stack_list[:-1]
                else:
                    current_buying[last_stock][0] -= 1

                # num_sell += 1
            if x not in current_buying:
                cur_stock = x
                cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
                today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
                current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
                num_buy +=1
            else:
                current_buying[x][0] += 1
                num_buy +=1
            # print(num_buy, current_buying)
            # input()

        # today_profit = 1
        # for cur in current_buying.values():
        #     num, start, end = cur
        #     profit = (end/start)**num
        #     today_profit *= profit
        # today_profit = today_profit**(1/num_buy)

        # if not np.isnan(today_profit):
        #     total_profit *= today_profit
        # total_profit = total_profit**(1/cnt)
        print("---------------------------------")
        print("day ", cnt, idx)
        print(num_buy, current_buying)
        # print("today_profit: ", today_profit)
        # print("total_profit: ", total_profit)
        print("current money: ", money)
        # print("------------------")
        # print(current_buying)
        # input()
        # clear_output()
    except:
        continue
    if cnt == 800:
        break

---------------------------------
day  1 2015-01-02 00:00:00
5 {'WELL': [1, 14.584031, 14.584031], 'PEAK': [1, 9.926856, 9.926856], 'BMY': [1, 1.78755, 1.78755], 'EW': [1, 15.538181, 15.538181], 'BDX': [1, 6.616302, 6.616302]}
current money:  100
---------------------------------
day  2 2015-01-05 00:00:00
10 {'WELL': [1, 14.584031, 14.710599], 'PEAK': [1, 9.926856, 9.994113], 'BMY': [2, 1.78755, 1.785444], 'EW': [1, 15.538181, 15.345455], 'BDX': [2, 6.616302, 6.603564], 'EA': [1, 2.23619, 2.23619], 'PDCO': [1, 4.908986, 4.908986], 'HOT': [1, 0.97561, 0.97561]}
current money:  100
---------------------------------
day  3 2015-01-06 00:00:00
15 {'WELL': [1, 14.584031, 14.896613], 'PEAK': [1, 9.926856, 10.207087], 'BMY': [3, 1.78755, 1.770706], 'EW': [1, 15.538181, 15.390304], 'BDX': [2, 6.616302, 6.674319], 'EA': [2, 2.23619, 2.231905], 'PDCO': [2, 4.908986, 4.920128], 'HOT': [1, 0.97561, 0.97561], 'ROST': [1, 24.003069, 24.003069], 'BBBY': [1, 4.405756, 4.405756]}
current money:  100
-

In [202]:
from IPython.display import clear_output
import numpy as np
import os
import time


max_result = 0
best = None
fee = 0.0007

for max_buying in range(1, 31):
    for top_k in range(1, 11):
        if max_buying < top_k:
            continue
        result = backtest(top_k, max_buying, fee)
        if result > max_result:
            max_result = result
            best = top_k, max_buying
print("best result:", best)

---------------------------------
config:  1 1
day  800 2018-03-07 00:00:00
1 {'CSRA': [1, 1.273041, 1.273041]}
current money:  540.4524597928363
time: 2.100587844848633
---------------------------------
config:  1 2
day  800 2018-03-07 00:00:00
2 {'CSRA': [2, 1.273354, 1.273041]}
current money:  402.0280592527657
time: 2.1224701404571533
---------------------------------
config:  2 2
day  800 2018-03-07 00:00:00
2 {'CSRA': [1, 1.270846, 1.273041], 'PXD': [1, 19.570301, 19.570301]}
current money:  682.4808854414977
time: 2.4594149589538574
---------------------------------
config:  1 3
day  800 2018-03-07 00:00:00
3 {'CSRA': [3, 1.273354, 1.273041]}
current money:  243.52584429237373
time: 2.2586331367492676
---------------------------------
config:  2 3
day  800 2018-03-07 00:00:00
3 {'CSRA': [2, 1.270846, 1.273041], 'PXD': [1, 19.570301, 19.570301]}
current money:  556.867039038147
time: 2.4587864875793457
---------------------------------
config:  3 3
day  800 2018-03-07 00:00:00
3 

In [225]:
backtest(5, 5, 0.0007, p=True)

---------------------------------
day  1 2015-01-02 00:00:00
---------------------------------
current stocks:  {'WELL': [1, 14.584031, 14.584031], 'PEAK': [1, 9.926856, 9.926856], 'BMY': [1, 1.78755, 1.78755], 'EW': [1, 15.538181, 15.538181], 'BDX': [1, 6.616302, 6.616302]}
current money:  100
time: 0.01377105712890625
---------------------------------
day  2 2015-01-05 00:00:00
---------------------------------
sell:  PEAK [1, 9.926856, 9.994113]
sell:  WELL [1, 14.584031, 14.710598]
sell:  EW [1, 15.538181, 15.345455]
sell:  BDX [1, 6.616302, 6.6035643]
sell:  BMY [2, 1.78755, 1.7854444]
current stocks:  {'BMY': [1, 1.78755, 1.7854444], 'EA': [1, 2.23619, 2.23619], 'PDCO': [1, 4.908986, 4.908986], 'BDX': [1, 6.603564, 6.603564], 'HOT': [1, 0.97561, 0.97561]}
current money:  99.858463225951
time: 0.023108243942260742
---------------------------------
day  3 2015-01-06 00:00:00
---------------------------------
sell:  HOT [1, 0.97561, 0.9756098]
sell:  BDX [1, 6.603564, 6.674319]
sell

793.706328978392

In [223]:
def backtest(top_k, max_buying, fee, p=False, early_stop=True):
    st = time.time()
    indexs = reset_df.index.drop_duplicates()
    cnt = 0 
    money = 100
    current_buying = {}
    num_buy = 0
    loss_cut = 0.95
    my_stack = None

    for idx in indexs:
        cnt+=1
        try:
            if p:
                print("---------------------------------")
                print("day ", cnt, idx)
                print("---------------------------------")
            if type(reset_df.loc[idx]['score']) is np.float32:
                stack = reset_df.loc[idx]
            else:
                stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)

            current_buying_list = list(current_buying.keys())
            for cur_stock in current_buying_list:
                today_stock_price = df.loc[cur_stock,idx][0].round(8)
                current_buying[cur_stock][2] = today_stock_price

            my_stack = stack[stack['instrument'].isin(current_buying_list)]
            my_stack = my_stack.sort_values(by='score' , ascending=True)
            my_stack_list = list(my_stack['instrument'])
            for x in stack['instrument'][:top_k]:
                if num_buy >= max_buying:
                    last_stock = my_stack_list[-1]
                    num_last_stock = current_buying[last_stock][0]

                    num_buy -= 1
                    start, end = current_buying[last_stock][1:]
                    if not (np.isnan(start) or np.isnan(end)):
                        earn = (end - start) / start * money / max_buying
                        earn -= 2 * fee * money / max_buying
                        if p:
                            print("sell: ", last_stock, current_buying[last_stock])
                        money += earn

                    if num_last_stock == 1:
                        del current_buying[last_stock]
                        my_stack_list = my_stack_list[:-1]
                    else:
                        current_buying[last_stock][0] -= 1

                if x not in current_buying:
                    cur_stock = x
                    today_stock_price = df.loc[cur_stock,idx][0].round(6)
                    current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
                else:
                    current_buying[x][0] += 1
                num_buy +=1
                # if p:
                #     print("buy: ", x, current_buying[x])
            if p:
                # print("config: ", top_k, max_buying)
                print("current stocks: ", current_buying)
                print("current money: ", money)
                print("time:", time.time() - st)
        except Exception as e:
            if early_stop and cnt >= 800:
                print("---------------------------------")
                print("config: ", top_k, max_buying)
                print("day ", cnt, idx)
                print(num_buy, current_buying)
                print("current money: ", money)
                print("time:", time.time() - st)
                break
            continue
        if early_stop and cnt >= 800:
            print("---------------------------------")
            print("config: ", top_k, max_buying)
            print("day ", cnt, idx)
            print(num_buy, current_buying)
            print("current money: ", money)
            print("time:", time.time() - st)
            break
    return money